In [22]:
class Node:
    def __init__(self,val):
        self.val = val
        self.to = {}

In [137]:
class Node:
    def __init__(self,val):
        self.val:int = val
        self.to:dict[Node,tuple[int,int]] = {} # destinationNode:(steps,price)
    
    def __str__(self) -> str:
        children = ','.join(str(i.val) for i in self.to.keys())
        return f"Node({self.val})"
    
    def __repr__(self) -> str:
        children = ','.join(str(i.val) for i in self.to.keys())
        return f"Node({self.val})"
    
    def full(self) -> str:
        children = ','.join(str(i.val) for i in self.to.keys())
        return f"Node({self.val})->[{children}]"

def update(node:Node, start:list[int]):
    # print("iter", node, start)
    if node.val in start:
        # print("found: ", node, " => ", start)
        return {}
    ret:dict[Node,set[tuple[int,int]]] = {
            i:set([tuple(node.to[i]),]) for i in node.to.keys()
        } # destinationNode:[(steps1,price1), (steps2,price2), ...]
    for destinationNode,(steps,price) in node.to.items():
        # print(f"step {node} to {destinationNode}")
        returned = update(destinationNode, [*start,node.val])
        # print(f"{node.val} going to {destinationNode.val} got {returned}")
        if returned == {}:
            # print(f"here on")
            ret[destinationNode].add((steps,price))
            continue
        for v,mylist in returned.items():
            # v is the a possible destination from our destination node
            # my list is a list of the steps and prices to that possible destination
            for (stp,prc) in mylist:
                newTuple = (stp+steps,prc+price)
                if ret.get(v):
                    ret[v].add(newTuple)
                else:
                    ret[v] = set([newTuple,])
    return ret

In [176]:
from cmath import inf

def findCheapestPrice(n: int, flights: list[list[int]], src: int, dst: int, k: int) -> int:
    nodes:dict[int,Node] = {}
    for s,d,p in flights:
        dnode = nodes.get(d)
        if dnode:
            snode = nodes.get(s)
            if snode:
                snode.to[dnode] = (1,p)
            else:
                nd = Node(s)
                nd.to[dnode] = (1,p)
                nodes[s] = nd
        else:
            snode = nodes.get(s)
            if snode:
                nd = Node(d)
                snode.to[nd] = (1,p)
                nodes[d] = nd
            else:
                nd1 = Node(s)
                nd2 = Node(d)
                nd1.to[nd2] = (1,p)
                nodes[s] = nd1
                nodes[d] = nd2
    for _,node in nodes.items():
        print(node.full())
    return method2(nodes, src, dst, k)

def method1(nodes:dict[int,Node], src:int, dst:int, k:int) -> int:
    results = {}
    for val,node in nodes.items():
        ret = update(node, [])
        results[val] = ret
    desired = results[src].get(nodes[dst])
    if not desired:
        return -1
    filtered = []
    k = k + 1
    for d in desired:
        if d[0] <= k:
            filtered.append(d)
    return min(filtered, key=lambda x:x[1])

def method2(nodes:dict[int,Node], src:int, dst:int, k:int) -> int:
    def recurse(node:Node, dst:int, k:int, visited:list[int]):
        results = []
        if k == 1:
            for nd in node.to.keys():
                if nd.val == dst:
                    return node.to[nd][1]
            return inf
        if node.val in visited:
            return inf
        for nd in node.to.keys():
            if nd.val == dst:
                results.append(node.to[nd][1])
            else:
                temp = recurse(nd, dst, k-1, [*visited, node.val]) + node.to[nd][1]
                results.append(temp)
        if len(results):
            return min(results)
        return inf
    
    k = k+1
    node = nodes[src]
    result = recurse(node, dst, k, [])
    if result == inf:
        return -1
    return result

In [157]:
findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 1)

100

In [178]:
findCheapestPrice(n = 4, flights = [[0,12,28],[5,6,39],[8,6,59],[13,15,7],[13,12,38],[10,12,35],[15,3,23],[7,11,26],[9,4,65],[10,2,38],[4,7,7],[14,15,31],[2,12,44],[8,10,34],[13,6,29],[5,14,89],[11,16,13],[7,3,46],[10,15,19],[12,4,58],[13,16,11],[16,4,76],[2,0,12],[15,0,22],[16,12,13],[7,1,29],[7,14,100],[16,1,14],[9,6,74],[11,1,73],[2,11,60],[10,11,85],[2,5,49],[3,4,17],[4,9,77],[16,3,47],[15,6,78],[14,1,90],[10,5,95],[1,11,30],[11,0,37],[10,4,86],[0,8,57],[6,14,68],[16,8,3],[13,0,65],[2,13,6],[5,13,5],[8,11,31],[6,10,20],[6,2,33],[9,1,3],[14,9,58],[12,3,19],[11,2,74],[12,14,48],[16,11,100],[3,12,38],[12,13,77],[10,9,99],[15,13,98],[15,12,71],[1,4,28],[7,0,83],[3,5,100],[8,9,14],[15,11,57],[3,6,65],[1,3,45],[14,7,74],[2,10,39],[4,8,73],[13,5,77],[10,0,43],[12,9,92],[8,2,26],[1,7,7],[9,12,10],[13,11,64],[8,13,80],[6,12,74],[9,7,35],[0,15,48],[3,7,87],[16,9,42],[5,16,64],[4,5,65],[15,14,70],[12,0,13],[16,14,52],[3,10,80],[14,11,85],[15,2,77],[4,11,19],[2,7,49],[10,7,78],[14,6,84],[13,7,50],[11,6,75],[5,10,46],[13,8,43],[9,10,49],[7,12,64],[0,10,76],[5,9,77],[8,3,28],[11,9,28],[12,16,87],[12,6,24],[9,15,94],[5,7,77],[4,10,18],[7,2,11],[9,5,41]], src = 13, dst = 4, k = 13)

Node(0)->[12,8,15,10]
Node(12)->[4,3,14,13,9,0,16,6]
Node(5)->[6,14,13,16,10,9,7]
Node(6)->[14,10,2,12]
Node(8)->[6,10,11,9,2,13,3]
Node(13)->[15,12,6,16,0,5,11,7,8]
Node(15)->[3,0,6,13,12,11,14,2]
Node(10)->[12,2,15,11,5,4,9,0,7]
Node(3)->[4,12,5,6,7,10]
Node(7)->[11,3,1,14,0,12,2]
Node(11)->[16,1,0,2,6,9]
Node(9)->[4,6,1,12,7,10,15,5]
Node(4)->[7,9,8,5,11,10]
Node(2)->[12,0,11,5,13,10,7]
Node(14)->[15,1,9,7,11,6]
Node(16)->[4,12,1,3,8,11,9,14]
Node(1)->[11,4,3,7]


47